In [10]:
import pickle
import numpy as np

def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

data_dir = "C:/Users/user/Desktop/birds/birds"
test_dir = data_dir + "/test"
test_text_embeddings = test_dir + "/char-CNN-RNN-embeddings.pickle"

text_embeddings = load_filenames(test_text_embeddings)

print(np.array(text_embeddings).shape)



(2933, 10, 1024)


In [20]:
import random

def load_embeddings(embeddings_file_path):
    """
    훈련된 텍스트 임베딩을 불러옴
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

data_dir = "C:/Users/user/Desktop/birds/birds"
test_dir = data_dir + "/test"
test_text_embeddings = test_dir + "/char-CNN-RNN-embeddings.pickle"

all_embeddings = load_embeddings(test_text_embeddings)

all_embeddings1 = all_embeddings[0, :, :]

print(all_embeddings1.shape)

embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
embedding = all_embeddings1[embedding_ix, :]

print(embedding.shape)


'''
원래 stackGAN 알고리즘에서 사용하는 텍스트 임베딩은 사전 훈련된 (1024,) 짜리 텍스트 임베딩 10개 중에서 임의로 한개를 선택하여
(1024,) 를 입력으로 사용하는 것임
'''

embeddings:  (2933, 10, 1024)
(10, 1024)
(1024,)


In [36]:
import csv
import numpy as np
import pickle

data_dir = "/home/csle/Desktop/CelebA_dataset"
attr_list = data_dir + "/list_attr_celeba_pure.csv"

def load_attr_list(csv_filename):
    read_list = []
    with open(csv_filename, 'r') as f:
        for line in f:
            read_list.append(line)
    return read_list  

attr_list = load_attr_list(attr_list)
# 각 이미지 속성 값 불러오기

def change_minus1_to_zero(attr_list):
    for i in range(len(attr_list)):
        attr_list[i] = attr_list[i].replace('-1', '0') 
        attr_list[i] = attr_list[i].replace('\n', '')
        attr_list[i] = attr_list[i].split(',')
        for n in range(40):
            attr_list[i][n] = int(attr_list[i][n]) 
    return attr_list
# 이미지 속성값을 문자열로 불러왔으므로 문자열에서 -1 을 0으로 바꾸고 \n를 제거한 후 정수리스트로 바꾸어 저장
    
attr_list = change_minus1_to_zero(attr_list)

def split_train_test(dataset, test_ratio):
    np.random.seed(1) # 항상 같은 train, test 세트가 생성됨 
    shuffled = np.random.permutation(len(dataset)) # 0에서 len(dataset)-1 까지 숫자의 무작위 배열을 만듬
    test_data_size = int(len(dataset) * test_ratio) # test_data 의 크기 결정
    test_data_index = shuffled[:test_data_size]
    train_data_index = shuffled[test_data_size:] # Dataframe 
    train_data = dataset.iloc[train_data_index]
    test_data = dataset.iloc[test_data_index] # .permutation 으로 만든 무작위 배열 index 에서 train, test 데이터를 Dataframe 으로 할당 함
    
    train_data = list(np.array(train_data[0].tolist()))
    test_data = list(np.array(test_data[0].tolist())) # (data_name) Dataframe -> list
    
    test_data_index = test_data_index.tolist()
    train_data_index = train_data_index.tolist() # (data_index) narray -> list
    
    return train_data, test_data, train_data_index, test_data_index

train_data_name, test_data_name, train_data_index, test_data_index = split_train_test(dataset, 0.2)

def parallelize(list, train_data_index, test_data_index):
    train_data_attr = []
    test_data_attr = []
    for i in train_data_index:
        train_data_attr.append(list[i])
    for n in test_data_index:
        test_data_attr.append(list[n])
    return train_data_attr, test_data_attr
# test, train 폴더에 무작위로 선택되어 분배 되어있는 이미지 데이터들에 대해, 
# 그 데이터들의 index 에 대응하는 자리에 속성값을 대응 시켜주기 위한 함수

train_data_attr, test_data_attr = parallelize(attr_list, train_data_index, test_data_index)

def save_pickle(save_data, name):
    with open(name, 'wb') as f:
        pickle.dump(save_data, f) # save_data 를 name 경로에 .pickle 파일로 저장함

save_pickle(train_data_attr, '/home/csle/Desktop/CelebA_dataset/train/attr_(embeddings).pickle') 
save_pickle(test_data_attr, '/home/csle/Desktop/CelebA_dataset/test/attr_(embeddings).pickle')        

'''
텍스트 임베딩 대신에 각 이미지의 속성값을 저장
'''

In [51]:
def load_embeddings(embeddings_file_path):
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
    return embeddings

filenames_test = load_embeddings('/home/csle/Desktop/CelebA_dataset/test/filenames.pickle')
print(type(filenames_test), np.array(filenames_test).shape)
filenames_train = load_embeddings('/home/csle/Desktop/CelebA_dataset/train/filenames.pickle')
print(type(filenames_train), np.array(filenames_train).shape)

list = load_attr_list('/home/csle/Desktop/CelebA_dataset/list_filenames.csv')
for i in range(len(list)):
    list[i] = list[i].replace('\n', '')

i = 0
for name in list:
    if name == filenames_test[3000]:
        break
    else:
        i += 1
print(i)

test_attr = load_embeddings('/home/csle/Desktop/CelebA_dataset/test/attr_(embeddings).pickle')
print(test_attr[3000])

attr_list = load_attr_list('/home/csle/Desktop/CelebA_dataset/list_attr_celeba_pure.csv')
attr_list[131452] = attr_list[131452].replace('-1', '0')
print(attr_list[131452])

'''
각 이미지의 속성값이 test, train 폴더의 이미지 파일의 인덱스와 잘 대응 됐는지 확인
'''

<class 'list'> (40519,)
<class 'list'> (162080,)
131452
[0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1]
0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1

